In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as sns
from matplotlib.ticker import AutoMinorLocator, MultipleLocator
from tqdm import tqdm
from sklearn.decomposition import PCA
from kneed import DataGenerator, KneeLocator
from sklearn.metrics import silhouette_score
from sklearn import metrics
from datetime import datetime
import time
plt.rcParams['font.family'] = "Times New Roman"
print(datetime.now())

2020-09-13 21:35:00.475508


In [2]:
#dataset loading
l1 = ["","RMSD_4ey4_1.csv","RMSD_4ey5_1.csv","RMSD_4ey6_1.csv","RMSD_4ey7_1.csv"]
l2 = ["","4ey4","4ey5","4ey6","4ey7"]
for i in range(1,5,1):
    globals()['d'+str(i)] = pd.read_csv(l1[i])
    globals()['X'+str(i)] = globals()['d'+str(i)].iloc[:,1:]


In [3]:
#PCA
file = open("./data/log.txt",'w')
pca = PCA(n_components = 5)
for i in range(1,5,1):
    globals()['X_'+str(i)] = pca.fit_transform(globals()['X'+str(i)])
    file.write(l1[i][0:-4])
    file.write(f'\nX_{str(i)}')
    file.write(f'\npca.explained_variance_ratio sum of 5 features : {(sum(pca.explained_variance_ratio_*100))}' )
    file.write(f'\nExplainedratio of indicidua  components : {pca.explained_variance_ratio_*100}')
    file.write('\n--------------------------------------------------------\n')


In [ ]:
#K_mean
from sklearn.cluster import KMeans 
fig, ax = plt.subplots()
plt.xlabel("Value of K") 
plt.ylabel("Sqaured Error (Cost)") 
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.xticks(np.arange(0, 25, step=1))
color =['black','red', 'blue', 'orange', 'green']
for i in tqdm(range(1,5,1)):
    globals()['cost_'+str(i)] =[] 
    for j in range(1, 26): 
        Km = KMeans(n_clusters = j) 
        Km.fit(globals()['X_'+str(i)]) 
        globals()['cost_'+str(i)].append(Km.inertia_)        
    time.sleep(0.5)
    
    plt.plot(range(1, 26), globals()['cost_'+str(i)], color = color[i], linewidth ='2')
    kneedle = KneeLocator(range(1, 26), globals()['cost_'+str(i)], S=1.0, curve="convex", direction="decreasing")
    file.write(l1[i][0:-4])
    file.write(f"\nThe selected value of K is {(round(kneedle.knee, 3))} \n")
    v1 = (round(kneedle.knee, 3))
    Km = KMeans(n_clusters = v1) 
    Km.fit(globals()['X_'+str(i)]) 
    labels = Km.labels_
    score1 = silhouette_score(globals()['X_'+str(i)], labels)
    score2 = metrics.calinski_harabasz_score(globals()['X_'+str(i)], labels)
    score3 = metrics.davies_bouldin_score(globals()['X_'+str(i)], labels)
    file.write(f"The slihoutte value for {v1} is silhoette score: {(round(score1, 3))}, calinski_harabasz score: {(round(score2, 3))},\
davies_bouldin score: {(round(score3, 3))} \n")\
    
plt.savefig('./data/kmean.jpeg', dpi = 600)
plt.show() # clear the plot 


file.close()    

In [ ]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=2)
for i in range(1,5,1):
    nbrs = neigh.fit(globals()['X_'+str(i)])
    distances, indices = nbrs.kneighbors(globals()['X_'+str(i)] )
    distances = np.sort(distances, axis=0)
    distances = distances[:,1]
    plt.plot(distances)
plt.savefig('knn.jpeg')
plt.show() # clear the plot 


In [ ]:
#DBSCAN
df2 = pd.DataFrame()
from sklearn.cluster import DBSCAN
list1 = ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan']
list2 = np.arange(0.10 ,1.6, 0.10)
list3 = [5,10,15,20,25,30,40,50]
for i1 in range(1,5,1):
    df2 = pd.DataFrame()
    for i in tqdm(list1):
        df3 = pd.DataFrame()
        for j in list2:
            for k in list3:
                clustering = DBSCAN(eps = j, min_samples = k, metric =i)
                labels = clustering.fit_predict(globals()['X_'+str(i1)])
                nCluster = len(set(labels))-(1 if -1 in labels else 0)
                n_noise = list(labels).count(-1)
                if nCluster == 1 or nCluster == 0:
                    df2 = df2.append({'Distance Metric': i, 'epsilon': round(j,2), 'Number of samples':int(k), 'Silhouette coefficient': -1, 'Number of clusters' : nCluster, "Noise": n_noise}, ignore_index = True)
                    df3 = df3.append({'Distance Metric': i, 'epsilon': round(j,2), 'Number of samples':int(k), 'Silhouette coefficient': -1, 'Number of clusters' : nCluster, "Noise": n_noise}, ignore_index = True)
                else:
                    score = silhouette_score(X1, labels, metric=i)
                    df2 = df2.append({'Distance Metric': i, 'epsilon': round(j,2), 'Number of samples':int(k), 'Silhouette coefficient': round(score,3), 'Number of clusters' : nCluster, "Noise": n_noise}, ignore_index = True)
                    df3 = df3.append({'Distance Metric': i, 'epsilon': round(j,2), 'Number of samples':int(k), 'Silhouette coefficient': round(score,3), 'Number of clusters' : nCluster, "Noise": n_noise}, ignore_index = True)
        df3_ = df3[['epsilon', 'Number of samples', 'Silhouette coefficient']]
        heatmap1_data = pd.pivot_table(df3_, values='Silhouette coefficient' ,index=['Number of samples'], columns='epsilon')
        sns.set()
        ob = sns.color_palette("pastel", as_cmap=True)
        #plt.figure(figsize = (4.0, 4.0))
        sns.heatmap(heatmap1_data, cmap= ob, annot=True, fmt=".2f", annot_kws={'size':7},\
             cbar=False, square=True,  linewidths=1, linecolor='white')
        plt.xlabel('epsilon', size = 11)
        plt.ylabel('Number of samples', size = 11)
        plt.xticks(fontsize=8)
        plt.yticks(fontsize=8,rotation='horizontal')
        plt.title(str(l2[i1]) + ' and ' + str(i), size = 12)
        plt.savefig('./data/{}_{}.jpeg'.format(l2[i1], i),dpi =600, format ='jpeg')
        plt.close()
    
    str1 = './data/'+l2[i1]+'_Dbscan.csv'
                    
    df2.to_csv(str1, mode ='w')

 17%|█████████████▊                                                                     | 1/6 [02:31<12:35, 151.17s/it]

In [ ]:
print(df2.nlargest(25,["Silhouette"]))

In [ ]:
#Agglomerative clustering
df3 = pd.DataFrame()
from sklearn.cluster import AgglomerativeClustering

list2 = [ 'euclidean', 'l1', 'l2', 'manhattan','cosine']
list3 = ['ward', 'average', 'complete', 'single']
for j in tqdm(list2):
    for k in list3:
        for i in range(2, 26,1):
            #print(i,j,k)
            if j == 'l1' or j == 'l2' or j == 'manhattan' or j == 'cosine' and k == 'ward':
                df3 = df3.append({'clusters': i, 'connectivity': j, 'linkage':k, 'Silhouette': -1}, ignore_index = True)
            else:
                clustering = AgglomerativeClustering(n_clusters= i, affinity = j, linkage= k)
                labels = clustering.fit_predict(X1)
            #nCluster = len(set(labels))-(1 if -1 in labels else 0)
            if len(np.unique(labels)) == 1:
                df3 = df3.append({'clusters': i, 'connectivity': j, 'linkage':k, 'Silhouette': -1}, ignore_index = True)
            else:
                score = silhouette_score(X1, labels, metric=j)
                df3 = df3.append({'clusters': i, 'connectivity': j, 'linkage':k,  'Silhouette': round(score,3)}, ignore_index = True)
                
df3.to_csv('sil3.csv', mode ='w')
print(df3.nlargest(20,["Silhouette"]))

In [ ]:
from sklearn.cluster import AffinityPropagation
sc = []
for i in tqdm([0.5,0.6,0.7,0.8,0.9, 0.95]):
    af = AffinityPropagation(damping = i, max_iter = 1000).fit(X1)
    cluster_centers_indices = af.cluster_centers_indices_
    labels = af.labels_
    if len(np.unique(labels)) == 1:
        pass
    else:
        score = round(silhouette_score(X1, labels),3)
        sc.append(score)
        nCluster = len(set(labels))-(1 if -1 in labels else 0)
        print(nCluster)
        print(score)

In [ ]:
#Spectral Clusterig
df3 = pd.DataFrame()
from sklearn.cluster import SpectralClustering

for i in tqdm(range(2, 26,1)):
    clustering = SpectralClustering(n_clusters=i,
        assign_labels="discretize",
        random_state=0).fit(X1)
    labels = clustering.labels_
    nCluster = len(set(labels))-(1 if -1 in labels else 0)
    score = silhouette_score(X1, labels, metric=j)
    df3 = df3.append({'clusters': nCluster, 'Silhouette': round(score,3)}, ignore_index = True)
                
df3.to_csv('sil_spc.csv', mode ='w')
print(df3.nlargest(20,["Silhouette"]))

In [ ]:
#meanshift Clusterig
df3 = pd.DataFrame()
from sklearn.cluster import MeanShift

for i in tqdm(np.arange(0.1,2,0.1)):
    clustering = MeanShift(bandwidth=i).fit(X1)
    labels = clustering.labels_
    nCluster = len(set(labels))-(1 if -1 in labels else 0)
    if nCluster >1 :
        score = silhouette_score(X1, labels, metric=j)
        df3 = df3.append({'clusters': nCluster, "bandwiidth": i, 'Silhouette': round(score,3)}, ignore_index = True)
    else: 
        pass
df3.to_csv('sil_mesh.csv', mode ='w')
print(df3.nlargest(20,["Silhouette"]))

In [ ]:
# plot the cost against K values 
cost = df3['Silhouette']

fig, ax = plt.subplots()
plt.plot(np.arange(0.1,2,0.1), cost, color ='b', linewidth ='3') 
plt.xlabel("Value of K") 
plt.ylabel("Sqaured Error (Cost)") 
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.xticks(np.arange(0.1,2, step = 0.1))
plt.show() # clear the plot 
kneedle = KneeLocator(np.arange(0.1,2,0.1), cost, S=1.0, curve="convex", direction="decreasing")
print("The selected value of K is {}".format(round(kneedle.knee, 3)))

In [ ]:
#BRICH clustering
df3 = pd.DataFrame()
from sklearn.cluster import Birch

for i in tqdm(range(2, 26,1)):
    clustering = Birch(n_clusters=i).fit(X1)
    labels = clustering.predict(X1)
    nCluster = len(set(labels))-(1 if -1 in labels else 0)
    score = silhouette_score(X1, labels, metric=j)
    df3 = df3.append({'clusters': nCluster, 'Silhouette': round(score,3)}, ignore_index = True)
                
df3.to_csv('sil_spc.csv', mode ='w')
print(df3.nlargest(20,["Silhouette"]))

In [ ]:
# plot the cost against K values 
cost = df3['Silhouette']

fig, ax = plt.subplots()
plt.plot(range(2,26,1), cost, color ='b', linewidth ='3') 
plt.xlabel("Value of K") 
plt.ylabel("Sqaured Error (Cost)") 
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.yaxis.set_ticks_position('left')
ax.xaxis.set_ticks_position('bottom')
plt.xticks(range(2,26,1))
plt.show() # clear the plot 
kneedle = KneeLocator(range(2,26,1), cost, S=1.0, curve="convex", direction="decreasing")


In [ ]:
# library & dataset
from matplotlib import pyplot as plt
import numpy as np

# create data
x = np.random.rand(15)
y = x+np.random.rand(15)
z = x+np.random.rand(15)
z=z*z

sns.color_palette("pastel", as_cmap = True)
# You can reverse it:
plt.scatter(x, y, s=z*2000, c=x,  alpha=0.4, edgecolors="grey", linewidth=2)



In [ ]:
str1 = './data/'+l2[i1]+'_Dbscan.csv'
print(str1)